In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import warnings
import time
warnings.filterwarnings('ignore')

def get_market_return(period_start, period_end, max_retries=3):
    """
    Get S&P 500 return for a given period with retry logic
    """
    for attempt in range(max_retries):
        try:
            # Convert dates to string format
            start_str = period_start.strftime('%Y-%m-%d')
            end_str = (period_end + timedelta(days=1)).strftime('%Y-%m-%d')

            # Explicit parameters
            sp500 = yf.Ticker('^GSPC')
            hist = sp500.history(start=start_str, end=end_str, auto_adjust=True)

            if not hist.empty and len(hist) >= 2:
                # Use Close prices instead of Adj Close
                start_price = hist['Close'].iloc[0]
                end_price = hist['Close'].iloc[-1]
                return ((end_price - start_price) / start_price) * 100
            else:
                print(f"Warning: Insufficient data for S&P 500 from {start_str} to {end_str}")
                return 0.0

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for market data: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2)  # Wait before retry
            else:
                print(f"Failed to fetch market data after {max_retries} attempts")
                return 0.0

def get_stock_returns(tickers, period_start, period_end, handle_gral=False):
    """
    Get stock returns for given tickers and period with improved error handling
    """
    returns = []

    # Convert dates to string format
    start_str = period_start.strftime('%Y-%m-%d')
    end_str = (period_end + timedelta(days=1)).strftime('%Y-%m-%d')

    for ticker in tickers:
        try:
            # Handle GRAL separately - only go back 3 quarters max
            if ticker == 'GRAL' and handle_gral:
                today = datetime.now()
                three_quarters_ago = today - timedelta(days=270)  # ~9 months
                if period_start < three_quarters_ago:
                    print(f"Skipping {ticker} - date range too far back")
                    returns.append(0.0)
                    continue

            # Create ticker object and get history
            stock_ticker = yf.Ticker(ticker)
            hist = stock_ticker.history(start=start_str, end=end_str, auto_adjust=True)

            if not hist.empty and len(hist) >= 2:
                # Calculate return using Close prices
                start_price = hist['Close'].iloc[0]
                end_price = hist['Close'].iloc[-1]
                stock_return = ((end_price - start_price) / start_price) * 100
                returns.append(stock_return)
                print(f"Successfully fetched {ticker}: {stock_return:.2f}%")
            else:
                print(f"Warning: Insufficient data for {ticker}")
                returns.append(0.0)

        except Exception as e:
            print(f"Error fetching {ticker}: {str(e)}")
            returns.append(0.0)
            time.sleep(0.5)  # Small delay to avoid rate limiting

    return np.array(returns)

def calculate_individual_stock_alpha():
    """
    Calculate alpha for individual stocks over 1Y, 2Y, and 3Y periods
    """
    # Stock tickers
    tickers = ['META', 'REGN', 'FI', 'GILD', 'MITK', 'FIS', 'IDCC', 'COUR', 'AMAT', 'KLAC',
               'NICE', 'WFC', 'AMD', 'LLY', 'NVO', 'DELL', 'BP', 'NOK', 'RMBS', 'EL',
               'UIS', 'ILMN', 'GOOG', 'SNPS', 'GRAL', 'UNH', 'GNRC', 'TXN', 'ROK']

    # Define time periods
    today = datetime.now()

    # Make sure we're not trying to fetch future data
    if today.weekday() >= 5:  # Weekend
        # Use last Friday
        days_back = today.weekday() - 4
        analysis_date = today - timedelta(days=days_back)
    else:
        analysis_date = today

    # Define periods
    one_year_start = analysis_date - timedelta(days=365)
    two_year_start = analysis_date - timedelta(days=730)
    three_year_start = analysis_date - timedelta(days=1095)

    # Display periods
    print(f"Analysis date: {analysis_date.strftime('%Y-%m-%d')}")
    print(f"1Y period: {one_year_start.strftime('%Y-%m-%d')} to {analysis_date.strftime('%Y-%m-%d')}")
    print(f"2Y period: {two_year_start.strftime('%Y-%m-%d')} to {analysis_date.strftime('%Y-%m-%d')}")
    print(f"3Y period: {three_year_start.strftime('%Y-%m-%d')} to {analysis_date.strftime('%Y-%m-%d')}")

    # Initialize return arrays
    returns_1y = np.zeros(len(tickers))
    returns_2y = np.zeros(len(tickers))
    returns_3y = np.zeros(len(tickers))

    # Fetch data in batches to avoid overwhelming the API
    print("\nFetching individual stock data...")

    # 1Y returns
    print("\nFetching 1Y returns...")
    returns_1y = get_stock_returns(tickers, one_year_start, analysis_date, handle_gral=True)

    # 2Y returns
    print("\nFetching 2Y returns...")
    returns_2y = get_stock_returns(tickers, two_year_start, analysis_date, handle_gral=True)

    # 3Y returns
    print("\nFetching 3Y returns...")
    returns_3y = get_stock_returns(tickers, three_year_start, analysis_date, handle_gral=True)

    # Get market returns
    print("\nFetching market data...")
    market_1y = get_market_return(one_year_start, analysis_date)
    market_2y = get_market_return(two_year_start, analysis_date)
    market_3y = get_market_return(three_year_start, analysis_date)

    print(f"\nMarket returns: 1Y={market_1y:.2f}%, 2Y={market_2y:.2f}%, 3Y={market_3y:.2f}%")

    # Risk-free rates (3-month T-bill = 4.42%)
    risk_free_1y = 4.42
    risk_free_2y = 4.42 * 2  # 2Y cumulative
    risk_free_3y = 4.42 * 3  # 3Y cumulative

    # Calculate alpha for each stock
    alphas_1y = []
    alphas_2y = []
    alphas_3y = []

    for i in range(len(tickers)):
        # 1Y Alpha
        alpha_1y = (returns_1y[i] - risk_free_1y) - (market_1y - risk_free_1y)
        alphas_1y.append(alpha_1y)

        # 2Y Alpha
        alpha_2y = (returns_2y[i] - risk_free_2y) - (market_2y - risk_free_2y)
        alphas_2y.append(alpha_2y)

        # 3Y Alpha
        alpha_3y = (returns_3y[i] - risk_free_3y) - (market_3y - risk_free_3y)
        alphas_3y.append(alpha_3y)

    # Create DataFrame
    df_comprehensive = pd.DataFrame({
        'Stock': tickers,
        '1Y_Return': returns_1y,
        '2Y_Return': returns_2y,
        '3Y_Return': returns_3y,
        '1Y_Alpha': alphas_1y,
        '2Y_Alpha': alphas_2y,
        '3Y_Alpha': alphas_3y
    })

    # Sort by alpha
    df_sorted_1y = df_comprehensive.sort_values('1Y_Alpha', ascending=False)
    df_sorted_2y = df_comprehensive.sort_values('2Y_Alpha', ascending=False)
    df_sorted_3y = df_comprehensive.sort_values('3Y_Alpha', ascending=False)

    # Save to CSV
    df_comprehensive.to_csv('alpha_comprehensive_analysis.csv', index=False)
    df_sorted_1y.to_csv('alpha_sorted_by_1y.csv', index=False)
    df_sorted_2y.to_csv('alpha_sorted_by_2y.csv', index=False)
    df_sorted_3y.to_csv('alpha_sorted_by_3y.csv', index=False)

    # OG CSV
    company_names = [
        'Meta Platforms, Inc.',
        'REGENERON PHARMACEUTICALS, INC.',
        'FISERV, INC.',
        'GILEAD SCIENCES, INC.',
        'MITEK SYSTEMS, INC.',
        'FIDELITY NATIONAL INFORMATION SERVICES, INC.',
        'InterDigital, Inc.',
        'COURSERA, INC.',
        'APPLIED MATERIALS, INC.',
        'KLA Corporation',
        'NICE LTD',
        'WELLS FARGO & COMPANY',
        'ADVANCED MICRO DEVICES, INC.',
        'ELI LILLY AND COMPANY',
        'Novo Nordisk A/S',
        'DELL TECHNOLOGIES INC.',
        'BP P.L.C.',
        'Nokia Oyj',
        'RAMBUS INC.',
        'THE ESTEE LAUDER COMPANIES INC.',
        'UNISYS CORPORATION',
        'ILLUMINA, INC.',
        'ALPHABET INC.',
        'SYNOPSYS, INC.',
        'GRAIL, INC.',
        'UNITEDHEALTH GROUP INCORPORATED',
        'GENERAC HOLDINGS INC.',
        'TEXAS INSTRUMENTS INCORPORATED',
        'ROCKWELL AUTOMATION, INC.'
    ]

    # Portfolio weights in original order
    portfolio_weights = [13.9407, 9.7876, 7.4736, 6.0693, 6.0428, 5.7082, 5.6708, 4.9494,
                        3.7019, 3.5918, 3.2948, 3.0735, 2.9403, 2.7125, 2.6726, 2.4349,
                        2.0997, 2.0104, 1.7872, 1.7506, 1.7250, 1.7083, 1.4630, 1.2205,
                        0.8973, 0.6003, 0.5892, 0.0836, 0.0000]

    # Create original order dataframe
    df_original_order = pd.DataFrame({
        'Company': company_names,
        'Ticker': tickers,
        'Weight_%': portfolio_weights,
        '1Y_Return_%': returns_1y,
        '1Y_Alpha_%': alphas_1y,
        '2Y_Return_%': returns_2y,
        '2Y_Alpha_%': alphas_2y,
        '3Y_Return_%': returns_3y,
        '3Y_Alpha_%': alphas_3y
    })

    # Save original order CSV
    df_original_order.to_csv('portfolio_original_order_alpha.csv', index=False)

    # Create a summary CSV with just the alpha values in original order
    df_alpha_summary = pd.DataFrame({
        'Ticker': tickers,
        'Company': company_names,
        'Weight_%': portfolio_weights,
        '1Y_Alpha': alphas_1y,
        '2Y_Alpha': alphas_2y,
        '3Y_Alpha': alphas_3y
    })

    df_alpha_summary.to_csv('alpha_summary_original_order.csv', index=False)

    return df_comprehensive, df_sorted_1y, df_sorted_2y, df_sorted_3y, df_original_order

def calculate_quarterly_portfolio_alpha():
    """
    Calculate portfolio alpha for the last 8 quarters
    """
    # Stock tickers
    tickers = ['META', 'REGN', 'FI', 'GILD', 'MITK', 'FIS', 'IDCC', 'COUR', 'AMAT', 'KLAC',
               'NICE', 'WFC', 'AMD', 'LLY', 'NVO', 'DELL', 'BP', 'NOK', 'RMBS', 'EL',
               'UIS', 'ILMN', 'GOOG', 'SNPS', 'GRAL', 'UNH', 'GNRC', 'TXN', 'ROK']

    # Portfolio weights
    weights = np.array([13.9407, 9.7876, 7.4736, 6.0693, 6.0428, 5.7082, 5.6708, 4.9494,
                       3.7019, 3.5918, 3.2948, 3.0735, 2.9403, 2.7125, 2.6726, 2.4349,
                       2.0997, 2.0104, 1.7872, 1.7506, 1.7250, 1.7083, 1.4630, 1.2205,
                       0.8973, 0.6003, 0.5892, 0.0836, 0.0000]) / 100

    # Build list of quarters
    quarters = []
    quarterly_returns = []
    market_returns = []

    today = datetime.now()

    # Define specific quarters to fetch
    quarter_dates = [
        ('Q2_2025', datetime(2025, 4, 1), datetime(2025, 6, 30)),
        ('Q1_2025', datetime(2025, 1, 1), datetime(2025, 3, 31)),
        ('Q4_2024', datetime(2024, 10, 1), datetime(2024, 12, 31)),
        ('Q3_2024', datetime(2024, 7, 1), datetime(2024, 9, 30)),
        ('Q2_2024', datetime(2024, 4, 1), datetime(2024, 6, 30)),
        ('Q1_2024', datetime(2024, 1, 1), datetime(2024, 3, 31)),
        ('Q4_2023', datetime(2023, 10, 1), datetime(2023, 12, 31)),
        ('Q3_2023', datetime(2023, 7, 1), datetime(2023, 9, 30))
    ]

    # Only include complete quarters
    valid_quarters = []
    for quarter_name, start_date, end_date in quarter_dates:
        if end_date < today:
            valid_quarters.append((quarter_name, start_date, end_date))

    print(f"\nFetching quarterly data for {len(valid_quarters)} quarters...")

    for quarter_name, start_date, end_date in valid_quarters:
        print(f"\nProcessing {quarter_name}...")

        # Get stock returns
        quarter_stock_returns = get_stock_returns(tickers, start_date, end_date, handle_gral=True)

        # Get market return
        quarter_market_return = get_market_return(start_date, end_date)

        quarters.append(quarter_name)
        quarterly_returns.append(quarter_stock_returns)
        market_returns.append(quarter_market_return)

    # Calculate portfolio metrics
    portfolio_alphas = []
    portfolio_returns = []
    risk_free_rate = 4.42 / 4  # Quarterly risk-free rate

    for i in range(len(quarters)):
        # Portfolio return
        portfolio_return = np.sum(weights * quarterly_returns[i])

        # Portfolio alpha
        portfolio_alpha = (portfolio_return - risk_free_rate) - (market_returns[i] - risk_free_rate)

        portfolio_returns.append(portfolio_return)
        portfolio_alphas.append(portfolio_alpha)

    # Create DataFrame
    df_quarterly = pd.DataFrame({
        'Quarter': quarters,
        'Portfolio_Return': portfolio_returns,
        'Market_Return': market_returns,
        'Portfolio_Alpha': portfolio_alphas
    })

    # Calculate rolling averages
    df_quarterly['Rolling_4Q_Alpha'] = df_quarterly['Portfolio_Alpha'].rolling(window=4, min_periods=1).mean()
    df_quarterly['Rolling_2Q_Alpha'] = df_quarterly['Portfolio_Alpha'].rolling(window=2, min_periods=1).mean()

    # Save to CSV
    df_quarterly.to_csv('quarterly_portfolio_alpha.csv', index=False)

    return df_quarterly

def calculate_ytd_portfolio_alpha():
    """
    Calculate YTD portfolio alpha
    """
    # Stock tickers and weights
    tickers = ['META', 'REGN', 'FI', 'GILD', 'MITK', 'FIS', 'IDCC', 'COUR', 'AMAT', 'KLAC',
               'NICE', 'WFC', 'AMD', 'LLY', 'NVO', 'DELL', 'BP', 'NOK', 'RMBS', 'EL',
               'UIS', 'ILMN', 'GOOG', 'SNPS', 'GRAL', 'UNH', 'GNRC', 'TXN', 'ROK']

    weights = np.array([13.9407, 9.7876, 7.4736, 6.0693, 6.0428, 5.7082, 5.6708, 4.9494,
                       3.7019, 3.5918, 3.2948, 3.0735, 2.9403, 2.7125, 2.6726, 2.4349,
                       2.0997, 2.0104, 1.7872, 1.7506, 1.7250, 1.7083, 1.4630, 1.2205,
                       0.8973, 0.6003, 0.5892, 0.0836, 0.0000]) / 100

    # YTD period
    today = datetime.now()
    ytd_start = datetime(today.year, 1, 1)

    print(f"\nCalculating YTD performance ({ytd_start.strftime('%Y-%m-%d')} to {today.strftime('%Y-%m-%d')})...")

    # Get returns
    ytd_returns = get_stock_returns(tickers, ytd_start, today, handle_gral=True)
    market_ytd = get_market_return(ytd_start, today)

    # Calculate portfolio return
    portfolio_ytd = np.sum(weights * ytd_returns)

    # Risk-free rate (prorated for YTD)
    days_elapsed = (today - ytd_start).days
    risk_free_ytd = 4.42 * (days_elapsed / 365)

    # Calculate alpha
    portfolio_alpha_ytd = (portfolio_ytd - risk_free_ytd) - (market_ytd - risk_free_ytd)

    print(f"\nYTD Performance Summary:")
    print(f"Portfolio Return: {portfolio_ytd:.2f}%")
    print(f"Market Return: {market_ytd:.2f}%")
    print(f"Portfolio Alpha: {portfolio_alpha_ytd:.2f}%")

    return portfolio_alpha_ytd, portfolio_ytd, market_ytd


if __name__ == "__main__":
    print("Portfolio Alpha Analysis with yFinance")
    print("=" * 70)

    try:
        # Individual stock analysis
        print("\n" + "=" * 70)
        print("INDIVIDUAL STOCK ALPHA ANALYSIS")
        print("=" * 70)

        df_comprehensive, df_sorted_1y, df_sorted_2y, df_sorted_3y, df_original_order = calculate_individual_stock_alpha()

        # Display results
        print("\nTop 10 Stocks by 1Y Alpha:")
        print(df_sorted_1y.head(10)[['Stock', '1Y_Return', '1Y_Alpha']].to_string(index=False, float_format='%.2f'))

        print("\nBottom 5 Stocks by 1Y Alpha:")
        print(df_sorted_1y.tail(5)[['Stock', '1Y_Return', '1Y_Alpha']].to_string(index=False, float_format='%.2f'))

        # Portfolio analysis
        print("\n" + "=" * 70)
        print("PORTFOLIO ALPHA ANALYSIS")
        print("=" * 70)

        # YTD analysis
        ytd_alpha, ytd_return, ytd_market = calculate_ytd_portfolio_alpha()

        # Quarterly analysis
        print("\n" + "=" * 70)
        print("QUARTERLY PORTFOLIO ALPHA")
        print("=" * 70)

        quarterly_df = calculate_quarterly_portfolio_alpha()

        if not quarterly_df.empty:
            print("\nQuarterly Performance:")
            print(quarterly_df.to_string(index=False, float_format='%.2f'))

            # Summary statistics
            print("\n" + "=" * 70)
            print("SUMMARY STATISTICS")
            print("=" * 70)

            if len(quarterly_df) >= 4:
                print(f"Trailing 4-Quarter Average Alpha: {quarterly_df['Rolling_4Q_Alpha'].iloc[-1]:.2f}%")
            if len(quarterly_df) >= 2:
                print(f"Trailing 2-Quarter Average Alpha: {quarterly_df['Rolling_2Q_Alpha'].iloc[-1]:.2f}%")

            print(f"Latest Quarter Alpha: {quarterly_df['Portfolio_Alpha'].iloc[-1]:.2f}%")

        # Save summary report
        summary_data = {
            'Metric': ['YTD Portfolio Return', 'YTD Market Return', 'YTD Portfolio Alpha',
                      'Latest Quarter Alpha', '4Q Rolling Alpha', '2Q Rolling Alpha'],
            'Value': [ytd_return, ytd_market, ytd_alpha,
                     quarterly_df['Portfolio_Alpha'].iloc[-1] if not quarterly_df.empty else 0,
                     quarterly_df['Rolling_4Q_Alpha'].iloc[-1] if len(quarterly_df) >= 4 else 0,
                     quarterly_df['Rolling_2Q_Alpha'].iloc[-1] if len(quarterly_df) >= 2 else 0]
        }

        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv('portfolio_summary.csv', index=False)



    except Exception as e:
        print(f"\nError during analysis: {str(e)}")
        print("Please ensure yFinance is up to date, or your internet is good enough.")